In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 16.4 MB/s eta 0:00:00


In [ ]:
import urllib
import zipfile
import random
import tensorflow as tf




In [ ]:
# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)


def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
    urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')
    with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
        zip_ref.extractall()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


callbacks = [
        # EarlyStopping(
        #     monitor='val_accuracy',
        #     min_delta=1e-4,
        #     patience=3,
        #     verbose=1
        # )
      #  ,
        ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
        # ,

        # ReduceLROnPlateau(monitor="val_loss",
        #                                         factor=0.2,  # multiply the learning rate by 0.2 (reduce by 5x)
        #                                         patience=2,
        #                                         verbose=1,  # print out when learning rate goes down
        #                                         min_lr=1e-7)
        ]



In [ ]:

download_and_extract_data()
IMG_SIZE = (128, 128)
BATCH_SIZE = 64
# The following code reads the training and validation data from their
# respective directories, resizes them into the specified image size
# and splits them into batches. You must fill in the image_size
# argument for both training and validation data.
# HINT: Image size is a tuple
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='train/',
  image_size=IMG_SIZE,
  label_mode='binary',
  shuffle=True,
  batch_size=BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='validation/',
  image_size= IMG_SIZE,
  label_mode='binary',
  batch_size=BATCH_SIZE,
  shuffle=False)

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [ ]:
def preprocess(image, label):
    image = image/255
    label = label/255

    image = tf.keras.applications.inception_v3.preprocess_input(image)


    return image, label



In [ ]:
train_ds = train_ds.map(
preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
tf.data.experimental.AUTOTUNE)
val_ds = val_ds.map(
preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds, val_ds





(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>,
 <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

data_augmentation = tf.keras.Sequential([
  layers.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.preprocessing.RandomRotation(0.2),
  layers.preprocessing.RandomZoom(height_factor=(0.2, 0.3), width_factor=(0.2, 0.3)),
  layers.preprocessing.RandomTranslation(0.3, 0.3, fill_mode='reflect', interpolation='bilinear',)
])

AttributeError: ignored

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    'train/',  # This is the source directory for training images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=1,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    'validation/',  # This is the source directory for training images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=1,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks = [
ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
      ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                                                 patience=3, min_lr=1e-5, verbose=1)
    ]

In [ ]:
train_ds, val_ds

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>,
 <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>)

In [ ]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop


#base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')



weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(128, 128, 3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
# print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    'train/',  # This is the source directory for training images
    target_size=(128, 128),  # All images will be resized to 150x150
    batch_size=64,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    'validation/',  # This is the source directory for training images
    target_size=(128, 128),  # All images will be resized to 150x150
    batch_size=64,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')





model.fit(
        train_ds,
        steps_per_epoch=8,
        epochs=100,
        verbose=1,
        validation_data=val_ds,
        validation_steps=8,
        callbacks=callbacks
    )


last layer output shape:  (None, 6, 6, 768)
Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/100
8/8 [==============================] - ETA: 0s - loss: 330.1458 - accuracy: 0.5137
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to mymodel.h5
8/8 [==============================] - 6s 387ms/step - loss: 330.1458 - accuracy: 0.5137 - val_loss: 1.0221e-17 - val_accuracy: 1.0000
Epoch 2/100
7/8 [=========================>....] - ETA: 0s - loss: 48.0024 - accuracy: 0.5714
Epoch 2: val_accuracy did not improve from 1.00000
8/8 [==============================] - 1s 88ms/step - loss: 45.0615 - accuracy: 0.5996 - val_loss: 1.5335 - val_accuracy: 0.9512
Epoch 3/100
7/8 [=========================>....] - ETA: 0s - loss: 30.8525 - accuracy: 0.7143
Epoch 3: val_accuracy did not improve from 1.00000
8/8 [==============================] - 1s 88ms/step - loss: 28.2347 - accuracy: 0.7246 - val_loss: 26.8000 - val_accuracy: 0.5410
Epoch 4/100



Epoch 20: val_accuracy did not improve from 1.00000
8/8 [==============================] - 1s 71ms/step - loss: 0.4275 - accuracy: 0.7941 - val_loss: 0.6959 - val_accuracy: 0.6172


In [ ]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.optimizers import RMSprop







# base_model = EfficientNetB7(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet', drop_connect_rate=0.4)

# for layer in base_model.layers:
#     layer.trainable = False



# model = tf.keras.Sequential([
#                                 base_model,
#                                 tf.keras.layers.GlobalAveragePooling2D(name="Layer1"),
#                                 tf.keras.layers.Dropout(0.4),
#                                 tf.keras.layers.Dense(1, activation='softmax')
#     ])
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['accuracy'], run_eagerly=True)



# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# validation_datagen = ImageDataGenerator(rescale=1 / 255)

# train_generator = train_datagen.flow_from_directory(
#     'train/',  # This is the source directory for training images
#     target_size=(128, 128),  # All images will be resized to 150x150
#     batch_size=64,
#     # Since we use binary_crossentropy loss, we need binary labels
#     class_mode='binary')

# validation_generator = validation_datagen.flow_from_directory(
#     'validation/',  # This is the source directory for training images
#     target_size=(128, 128),  # All images will be resized to 150x150
#     batch_size=64,
#     # Since we use binary_crossentropy loss, we need binary labels
#     class_mode='binary')


#train_ds, val_ds





In [ ]:
model.fit(
        train_ds,
        steps_per_epoch=100,
        epochs=10,
        verbose=1,
        validation_data=val_ds,
        validation_steps=8,
        callbacks=callbacks
    )

ValueError: ignored

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model

def create_model():
  base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
  # for layer in base_model.layers[:-30]:
  #   layer.trainable = False
  augmented = data_augmentation(base_model)
  resnet = base_model(augmented)
  pooling = layers.GlobalAveragePooling2D()(resnet)
  dropout = layers.Dropout(0.4)(pooling)
  outputs = Dense(len(class_types), activation="softmax")(dropout)
  model = Model(inputs=inputs, outputs=outputs)

  return model

model = create_model()
model.summary()

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              metrics=['acc'])



history = model.fit(train_ds,
                    epochs = 60,
                    validation_data=val_ds,
                    callbacks=callbacks)


ValueError: ignored